# TO_CSV - Converts interaction logs into frequency histograms

* Runs on individual files
* Pulls data from newData/Dataset_x
* Outputs data to newCSV/Dataset_x

In [1]:
### Import necessary modules
import json
import re
import numpy as np
import csv

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [2]:
### Select participant and dataset
PID = 'p8'
dataset = '4'

filename = 'newData/Dataset_' + dataset + '/' + PID + '.json'
tfidf = 'newData/tfidf_d' + dataset + '.json'

In [3]:
### Open file and pre-process data
f = open(tfidf)

TFIDFVecs = json.load(f)

histograms = TFIDFVecs['histograms']
vocabulary = TFIDFVecs['vocabulary']
titles = TFIDFVecs['titles']

userData = json.load(open(filename))['data']

# docTypes = ['Doc_open', 'Reading']
docTypes = ['Reading']
createTypes = ['Search', 'Highlight', 'Connection', 'Add Note']

In [4]:
# check_str = "H4BuKWENpHH3uYy"
# print(len(titles))
# temp = '\t'.join(titles)
# print(len(temp))
# res = check_str in temp
# print(res)



In [19]:
### Process logs
outVecs = []
outTimes = []
numFailed = 0
counter = 0
for data in userData:
    if data['InteractionType'] in docTypes:
        docTitle = data['ID']
        docTime = data['time']
        if docTitle in titles:
            # print("FoundDoc titled: " + str(docTitle))
            docPos = titles.index(docTitle)
            outVecs.append(histograms[docPos])
            outTimes.append(docTime)
            # print("Adding a historgram of length " + str(len(histograms[docPos])) + " to outVecs")
            # print("Outvecs is now: "+ str(len(outVecs))+" items long")
        else:
            print("XX- Cannot find Title in dataset: "+str(docTitle))
            numFailed = numFailed + 1
    if data['InteractionType'] in createTypes:
        string = data['Text']
        docTime = data['time']
        words = re.findall(r'\w+', string)
        tempVec = np.zeros(len(vocabulary), dtype = int)
        changed = 0
        numAdded = 0
        posAdded = []
        for word in words:
            word = word.lower() #set the words to lower case before checking the topics.
            if word in vocabulary:
                wordPos = vocabulary.index(word)
                tempVec[wordPos] = tempVec[wordPos] + 1
                numAdded = numAdded + 1
                changed = 1
                if wordPos not in posAdded:
                    posAdded.append(wordPos)
            if changed == 0:
                print("XX- Cannot find word in dataset: "+str(word))
                numFailed = numFailed + 1
            else:
                for position in posAdded:
                    tempVec[position] = tempVec[position] / numAdded
                # sumWordsInEvent = 0
                # for wordFreq in tempVec:
                #     sumWordsInEvent = sumWordsInEvent + tempVec[wordFreq]
                # # print(sumWordsInEvent)
                # if (sumWordsInEvent > 0):
                outVecs.append(tempVec.tolist())
                outTimes.append(docTime)
#should print 0                
print("failed to find " + str(numFailed) + " references")


XX- Cannot find word in dataset: nugent
XX- Cannot find word in dataset: nugent
XX- Cannot find word in dataset: blount
failed to find 3 references


In [20]:
### Export data to two files
outFilename = 'wordsCSV/Dataset_' + dataset + '/' + PID + '.csv'
f = open(outFilename, 'w', newline='')

writer = csv.writer(f)
header = vocabulary.copy()
header.insert(0, 'intTime')
writer.writerow(header)

for i in range(len(outVecs)):
    row = outVecs[i].copy()
    row.insert(0, outTimes[i])
    writer.writerow(row)
f.close()


outFilename = 'newCSV/Dataset_' + dataset + '/' + PID + '.csv'
f = open(outFilename, 'w', newline='')

writer = csv.writer(f)
# header = vocabulary.copy()
# header.insert(0, 'intTime')
# writer.writerow(header)

for i in range(len(outVecs)):
    row = outVecs[i].copy()
    row.insert(0, outTimes[i])
    writer.writerow(row)
f.close()